In [ ]:
import jax
import jax.numpy as jnp
import jax.tree_util as jtu
import time
import os
from dataclasses import dataclass
from collections import namedtuple
import pyro
import optax
from pyro.infer import SVI, TraceGraph_ELBO
import pyro.distributions as dist
import pyro.poutine as poutine
import pyro.contrib.examples.multi_mnist as multi_mnist
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.font_manager as font_manager
from matplotlib.patches import Rectangle
import matplotlib.ticker as ticker
import matplotlib.patches as patches
import seaborn as sns
from matplotlib import cm
from matplotlib.ticker import LinearLocator
from matplotlib import rcParams
from scipy.interpolate import griddata
import genjax
import adevjax
from genjax import grasp

key = jax.random.PRNGKey(314159)
console = genjax.pretty()
sns.set_theme(style="white")
# font_path = (
#     "/home/femtomc/.local/share/fonts/Unknown Vendor/TrueType/Lato/Lato_Bold.ttf"
# )
# font_manager.fontManager.addfont(font_path)
# custom_font_name = font_manager.FontProperties(fname=font_path).get_name()
# rcParams["font.family"] = custom_font_name

console = genjax.pretty()
key = jax.random.PRNGKey(314159)
label_fontsize = 70  # Set the desired font size here

smoke_test = "CI" in os.environ
assert pyro.__version__.startswith("1.8.6")

In [ ]:
inpath = "./data/air/.data"
X_np, Y = multi_mnist.load(inpath)
X_np = X_np.astype(np.float32)
X_np /= 255.0
mnist = jnp.array(X_np)
true_counts = jnp.array([len(objs) for objs in Y])


def show_images(imgs):
    fig = plt.figure(figsize=(8, 2))
    for i, img in enumerate(imgs):
        ax = plt.subplot(1, len(imgs), i + 1)
        ax.set_xticks([])
        ax.set_yticks([])
        plt.imshow(img, cmap="gray_r")


show_images(mnist[9:14])

## Defining the variational ingredients

### Utilities / learnable pieces

In [ ]:
from genjax import Pytree
import equinox as eqx
from genjax.typing import Any
from genjax.typing import Tuple
from genjax.typing import FloatArray
from genjax.typing import Int
from genjax.typing import IntArray
from genjax.typing import PRNGKey
from genjax.typing import typecheck

# Utilities for defining the model and the guide.


@dataclass
class Decoder(Pytree):
    dense_1: Any
    dense_2: Any

    def flatten(self):
        return (self.dense_1, self.dense_2), ()

    @classmethod
    def new(cls, key1, key2):
        dense_1 = eqx.nn.Linear(50, 200, key=key1)
        dense_2 = eqx.nn.Linear(200, 400, key=key2)
        return Decoder(dense_1, dense_2)

    def __call__(self, z_what):
        v = self.dense_1(z_what)
        v = jax.nn.leaky_relu(v)
        v = self.dense_2(v)
        return jax.nn.sigmoid(v)


# Create our decoder.
key, sub_key1, sub_key2 = jax.random.split(key, 3)
decoder = Decoder.new(sub_key1, sub_key2)

# Create our RNN for the guide.
key, sub_key = jax.random.split(key)
rnn = eqx.nn.LSTMCell(2554, 256, key=sub_key)

# Create our baseline RNN and predict layer for the guide.
key, sub_key = jax.random.split(key)
bl_rnn = eqx.nn.LSTMCell(2554, 256, key=sub_key)
key, sub_key = jax.random.split(key)
bl_predict = eqx.nn.Linear(256, 1, key=sub_key)


@dataclass
class Encoder(Pytree):
    dense_1: Any
    dense_2: Any

    def flatten(self):
        return (self.dense_1, self.dense_2), ()

    @classmethod
    def new(cls, key1, key2):
        dense_1 = eqx.nn.Linear(400, 200, key=key1)
        dense_2 = eqx.nn.Linear(200, 100, key=key2)
        return Encoder(dense_1, dense_2)

    def __call__(self, data):
        v = self.dense_1(data)
        v = jax.nn.leaky_relu(v)
        v = self.dense_2(v)
        return v[0:50], jax.nn.softplus(v[50:])


key, sub_key1, sub_key2 = jax.random.split(key, 3)
encoder = Encoder.new(sub_key1, sub_key2)


@dataclass
class Predict(Pytree):
    dense: Any

    def flatten(self):
        return (self.dense,), ()

    @classmethod
    def new(cls, key):
        dense = eqx.nn.Linear(256, 7, key=key)
        return Predict(dense)

    def __call__(self, h):
        a = self.dense(h)
        z_pres_p = jax.nn.sigmoid(a[0:1])
        z_where_loc = a[1:4]
        z_where_scale = jax.nn.softplus(a[4:])
        return z_pres_p, z_where_loc, z_where_scale


key, sub_key = jax.random.split(key)
predict = Predict.new(key)

In [ ]:
#######
# STN #
#######

# modified from https://github.com/kevinzakka/spatial-transformer-network/blob/master/stn/transformer.py


def affine_grid_generator(height, width, theta):
    """
    This function returns a sampling grid, which when
    used with the bilinear sampler on the input feature
    map, will create an output feature map that is an
    affine transformation [1] of the input feature map.

    Input
    -----
    - height: desired height of grid/output. Used
      to downsample or upsample.

    - width: desired width of grid/output. Used
      to downsample or upsample.

    - theta: affine transform matrices of shape (num_batch, 2, 3).
      For each image in the batch, we have 6 theta parameters of
      the form (2x3) that define the affine transformation T.

    Returns
    -------
    - normalized grid (-1, 1) of shape (num_batch, 2, H, W).
      The 2nd dimension has 2 components: (x, y) which are the
      sampling points of the original image for each point in the
      target image.

    Note
    ----
    [1]: the affine transformation allows cropping, translation,
         and isotropic scaling.
    """
    num_batch = theta.shape[0]

    # create normalized 2D grid
    x = jnp.linspace(-1.0, 1.0, width)
    y = jnp.linspace(-1.0, 1.0, height)
    x_t, y_t = jnp.meshgrid(x, y)

    # flatten
    x_t_flat = jnp.reshape(x_t, [-1])
    y_t_flat = jnp.reshape(y_t, [-1])

    # reshape to [x_t, y_t , 1] - (homogeneous form)
    ones = jnp.ones_like(x_t_flat)
    sampling_grid = jnp.stack([x_t_flat, y_t_flat, ones])

    # repeat grid num_batch times
    sampling_grid = jnp.expand_dims(sampling_grid, axis=0)
    sampling_grid = jnp.tile(sampling_grid, [num_batch, 1, 1])

    # transform the sampling grid - batch multiply
    batch_grids = jnp.matmul(theta, sampling_grid)
    # batch grid has shape (num_batch, 2, H*W)

    # reshape to (num_batch, 2, H, W)
    batch_grids = jnp.reshape(batch_grids, [num_batch, 2, height, width])

    return batch_grids


def bilinear_sampler(img, x, y):
    """
    Performs bilinear sampling of the input images according to the
    normalized coordinates provided by the sampling grid. Note that
    the sampling is done identically for each channel of the input.

    To test if the function works properly, output image should be
    identical to input image when theta is initialized to identity
    transform.

    Input
    -----
    - img: batch of images in (B, H, W, C) layout.
    - grid: x, y which is the output of affine_grid_generator.

    Returns
    -------
    - out: interpolated images according to grids. Same size as grid.
    """
    H = jnp.shape(img)[1]
    W = jnp.shape(img)[2]
    max_y = H - 1
    max_x = W - 1
    zero = jnp.zeros([], dtype=int)

    # rescale x and y to [0, W-1/H-1]
    x = 0.5 * ((x + 1.0) * max_x - 1)
    y = 0.5 * ((y + 1.0) * max_y - 1)

    # grab 4 nearest corner points for each (x_i, y_i)
    x0 = jnp.floor(x).astype(int)
    x1 = x0 + 1
    y0 = jnp.floor(y).astype(int)
    y1 = y0 + 1

    # clip to range [0, H-1/W-1] to not violate img boundaries
    x0 = jnp.clip(x0, zero, max_x)
    x1 = jnp.clip(x1, zero, max_x)
    y0 = jnp.clip(y0, zero, max_y)
    y1 = jnp.clip(y1, zero, max_y)

    # get pixel value at corner coords
    Ia = get_pixel_value(img, x0, y0)
    Ib = get_pixel_value(img, x0, y1)
    Ic = get_pixel_value(img, x1, y0)
    Id = get_pixel_value(img, x1, y1)

    # recast as float for delta calculation
    x0 = x0.astype(float)
    x1 = x1.astype(float)
    y0 = y0.astype(float)
    y1 = y1.astype(float)

    # calculate deltas
    wa = (x1 - x) * (y1 - y)
    wb = (x1 - x) * (y - y0)
    wc = (x - x0) * (y1 - y)
    wd = (x - x0) * (y - y0)

    # add dimension for addition
    wa = jnp.expand_dims(wa, axis=3)
    wb = jnp.expand_dims(wb, axis=3)
    wc = jnp.expand_dims(wc, axis=3)
    wd = jnp.expand_dims(wd, axis=3)

    # compute output
    out = wa * Ia + wb * Ib + wc * Ic + wd * Id
    return out


def get_pixel_value(img, x, y):
    """
    Utility function to get pixel value for coordinate
    vectors x and y from a  4D tensor image.

    Input
    -----
    - img: tensor of shape (B, H, W, C)
    - x: flattened tensor of shape (B*H*W,)
    - y: flattened tensor of shape (B*H*W,)

    Returns
    -------
    - output: tensor of shape (B, H, W, C)
    """
    batch_size, height, width = jnp.shape(x)

    batch_idx = jnp.arange(0, batch_size)
    batch_idx = jnp.reshape(batch_idx, (batch_size, 1, 1))
    b = jnp.tile(batch_idx, (1, height, width))

    indices = jnp.stack([b, y, x], 3)

    return gather_nd(img, indices)


# from: https://github.com/google/jax/discussions/6119
def gather_nd_unbatched(params, indices):
    return params[tuple(jnp.moveaxis(indices, -1, 0))]


def gather_nd(params, indices, batch=False):
    if not batch:
        return gather_nd_unbatched(params, indices)
    else:
        return vmap(gather_nd_unbatched, (0, 0), 0)(params, indices)


def expand_z_where(z_where):
    # Takes 3-dimensional vectors, and massages them into 2x3 matrices with elements like so:
    # [s,x,y] -> [[s,0,x],
    #             [0,s,y]]
    n = 1
    expansion_indices = jnp.array([1, 0, 2, 0, 1, 3])
    z_where = jnp.expand_dims(z_where, axis=0)
    out = jnp.concatenate((jnp.broadcast_to(jnp.zeros([1, 1]), (n, 1)), z_where), 1)
    return jnp.reshape(out[:, expansion_indices], (n, 2, 3))


def object_to_image(z_where, obj):
    n = 1
    theta = expand_z_where(z_where)
    grid = affine_grid_generator(50, 50, theta)
    x_s = grid[:, 0, :, :]
    y_s = grid[:, 1, :, :]
    out = bilinear_sampler(jnp.reshape(obj, (n, 20, 20, 1)), x_s, y_s)
    return jnp.reshape(out, (50, 50))


def z_where_inv(z_where):
    # Take a batch of z_where vectors, and compute their "inverse".
    # That is, for each row compute:
    # [s,x,y] -> [1/s,-x/s,-y/s]
    # These are the parameters required to perform the inverse of the
    # spatial transform performed in the generative model.
    n = 1
    out = jnp.array([1, *(-z_where[1:])])
    out = out / z_where[0]
    return out


def image_to_object(z_where, image):
    n = 1
    theta_inv = expand_z_where(z_where_inv(z_where))
    grid = affine_grid_generator(20, 20, theta_inv)
    x_s = grid[:, 0, :, :]
    y_s = grid[:, 1, :, :]
    out = bilinear_sampler(jnp.reshape(image, (n, 50, 50, 1)), x_s, y_s)
    return jnp.reshape(out, (400,))

### Model

In [ ]:
#########
# Model #
#########

# Fixed constants.
z_where_prior_loc = jnp.array([3.0, 0.0, 0.0])
z_where_prior_scale = jnp.array([0.2, 1.0, 1.0])
z_what_prior_loc = jnp.zeros(50, dtype=float)
z_what_prior_scale = jnp.ones(50, dtype=float)
z_pres_prior = -4.59512


@genjax.gen
@typecheck
def step(
    t: Int,
    decoder: Decoder,
    prev_x: FloatArray,
    prev_z_pres: IntArray,
):
    z_pres = grasp.flip_reinforce(z_pres_prior * (2 - prev_z_pres[0])) @ f"z_pres_{t}"
    z_pres = jnp.array([z_pres.astype(int)])
    z_where = (
        grasp.mv_normal_diag_reparam(z_where_prior_loc, z_where_prior_scale)
        @ f"z_where_{t}"
    )
    z_what = (
        grasp.mv_normal_diag_reparam(z_what_prior_loc, z_what_prior_scale)
        @ f"z_what_{t}"
    )
    y_att = decoder(z_what)
    y = object_to_image(z_where, y_att)
    x = prev_x + (y * z_pres)
    return x, z_pres


# TODO: Make sure that this works, where t is a static int.
@genjax.gen
@typecheck
def model(decoder: Decoder):
    x = jnp.zeros((50, 50), dtype=float)
    z_pres = jnp.ones(1, dtype=int)
    for t in range(3):
        x, z_pres = step.inline(t, decoder, x, z_pres)
    obs = grasp.mv_normal_diag_reparam(x, 0.3 * jnp.ones_like(x)) @ "obs"
    return x

#### Samples from the model

In [ ]:
tr = jax.jit(model.simulate)(key, (decoder,))
tr.strip()

### Guide

In [ ]:
#########
# Guide #
#########


def baseline_step(bl_rnn, bl_predict, img_arr, prev):
    prev_bl_h, prev_bl_c, prev_z_where, prev_z_what, prev_z_pres = prev
    rnn_input = jnp.concatenate([img_arr, prev_z_where, prev_z_what, prev_z_pres])
    bl_h, bl_c = bl_rnn(rnn_input, (prev_bl_h, prev_bl_c))
    bl_value = bl_predict(bl_h) * prev_z_pres[0]
    return bl_value, bl_h, bl_c


@genjax.gen
@typecheck
def guide_step(
    t: Int,
    rnn: eqx.nn.LSTMCell,
    encoder: Encoder,
    predict: Predict,
    bl_rnn: eqx.nn.LSTMCell,
    bl_predict: eqx.nn.Linear,
    img_arr,
    prev: Tuple,
):
    (
        prev_bl_h,
        prev_bl_c,
        prev_z_where,
        prev_z_what,
        prev_z_pres,
        prev_h,
        prev_c,
    ) = prev

    # Baseline step.
    bl_value, bl_h, bl_c = baseline_step(
        bl_rnn,
        bl_predict,
        img_arr,
        (prev_bl_h, prev_bl_c, prev_z_where, prev_z_what, prev_z_pres),
    )
    rnn_input = jnp.concatenate([img_arr, prev_z_where, prev_z_what, prev_z_pres])
    h, c = rnn(rnn_input, (prev_h, prev_c))
    z_pres_p, z_where_loc, z_where_scale = predict(h)

    # Baseline applied here.
    #z_pres = (
    #    grasp.baseline(grasp.flip_reinforce)(
    #        bl_value[0], z_pres_p[0] * (2 - prev_z_pres[0])
    #    )
    #    @ f"z_pres_{t}"
    #)
    z_pres = grasp.flip_enum(0.01) @ f"z_pres_{t}"
    # z_pres = grasp.flip_reinforce(z_pres_p[0] * (2 - prev_z_pres[0])) @ f"z_pres_{t}"

    z_pres = jnp.array([z_pres.astype(int)])
    z_where = grasp.mv_normal_diag_reparam(z_where_loc, z_where_scale) @ f"z_where_{t}"
    x_att = image_to_object(z_where, img_arr)
    z_what_loc, z_what_scale = encoder(x_att)
    z_what = grasp.mv_normal_diag_reparam(z_what_loc, z_what_scale) @ f"z_what_{t}"
    return bl_h, bl_c, z_where, z_what, z_pres, h, c, bl_value


@genjax.gen
@typecheck
def guide(
    data: genjax.ChoiceMap,
    rnn: eqx.nn.LSTMCell,
    encoder: Encoder,
    predict: Predict,
    bl_rnn: eqx.nn.LSTMCell,
    bl_predict: eqx.nn.Linear,
):
    bl_h = jnp.zeros(256)
    bl_c = jnp.zeros(256)
    z_where = jnp.zeros(3)
    z_what = jnp.zeros(50)
    z_pres = jnp.ones(1)
    h = jnp.zeros(256)
    c = jnp.zeros(256)
    img = data["obs"]
    img_arr = img.flatten()
    bl_values = []

    for t in range(3):
        (bl_h, bl_c, z_where, z_what, z_pres, h, c, bl_value) = guide_step.inline(
            t,
            rnn,
            encoder,
            predict,
            bl_rnn,
            bl_predict,
            img_arr,
            (bl_h, bl_c, z_where, z_what, z_pres, h, c),
        )
        bl_values.append(bl_value)

    return bl_values

#### Samples from the guide

In [ ]:
data_chm = genjax.choice_map({"obs": jnp.ones((50, 50))})
tr = jax.jit(guide.simulate)(key, (data_chm, rnn, encoder, predict, bl_rnn, bl_predict))
tr.strip()

## Training

### Make sure grads are working

#### Define ELBO objective

In [ ]:
data = genjax.choice_map({"obs": jnp.ones((50, 50))})
objective = grasp.elbo(model, guide, data)

#### Go go grads

In [ ]:
jitted = jax.jit(objective.value_and_grad_estimate)
loss, ((decoder_grads,), (_, rnn_grads, encoder_grads, predict_grads, _, _)) = jitted(
    key, ((decoder,), (data, rnn, encoder, predict, bl_rnn, bl_predict))
)

In [ ]:
loss

### Dataloader

In [ ]:
def data_loader(
    data,
    batch_size,
    shuffle=True,
):
    N = len(data)
    data_idxs = np.arange(N)
    num_batch = int(np.ceil(N // batch_size))

    def init():
        return (
            num_batch,
            np.random.permutation(data_idxs) if shuffle else data_idxs,
        )

    def get_batch(i=0, idxs=data_idxs):
        ret_idx = jax.lax.dynamic_slice_in_dim(idxs, i * batch_size, batch_size)
        return jax.lax.index_take(data, (ret_idx,), axes=(0,))

    return init, get_batch

In [ ]:
##################
# Count Accuracy #
##################


def count_accuracy(data, true_counts, guide, batch_size=1000):
    global prng_key
    assert data.shape[0] == true_counts.shape[0], "Size mismatch."
    assert data.shape[0] % batch_size == 0, "Input size must be multiple of batch_size."

    def eval_guide(key, data, params):
        data_chmp = genjax.choice_map({"obs": data})
        return guide.simulate(key, (data_chmp, *params))

    batch_eval_guide = jax.jit(jax.vmap(eval_guide, in_axes=(0, 0, None)))

    @jax.jit
    def evaluate_count_accuracy(key, params):
        def evaluate_batch(counts, batch_id):
            data_batch = jax.lax.dynamic_slice_in_dim(
                data, batch_id * batch_size, batch_size
            )
            true_counts_batch = jax.lax.dynamic_slice_in_dim(
                true_counts, batch_id * batch_size, batch_size
            )
            data_chmp = genjax.choice_map({"obs": data_batch})
            # evaluate guide
            keys = jax.random.split(jax.random.fold_in(key, batch_id), batch_size)
            tr = batch_eval_guide(keys, data_batch, params)
            z_where = [tr[f"z_where_{i}"] for i in range(3)]
            z_pres = [tr[f"z_pres_{i}"] for i in range(3)]
            # compute stats
            inferred_counts = sum(z for z in z_pres)
            true_counts_m = jax.nn.one_hot(true_counts_batch, 3)
            inferred_counts_m = jax.nn.one_hot(inferred_counts, 4)
            counts += (true_counts_m.T @ inferred_counts_m).astype(int)
            error_ind = 1 - (true_counts_batch == inferred_counts).astype(int)
            # error_ix = error_ind.nonzero()[0]
            # error_latent = jnp.take(latents_to_tensor((z_where, z_pres)), error_ix, 0)
            return counts, error_ind

        counts = jnp.zeros((3, 4), dtype=int)
        counts, error_indices = jax.lax.scan(
            evaluate_batch, counts, jnp.arange(data.shape[0] // batch_size)
        )

        acc = jnp.sum(jnp.diag(counts)).astype(float) / data.shape[0]
        error_indices = jnp.concatenate(
            error_indices
        )  # .nonzero()[0]  # <- not JIT compilable
        return acc, counts, error_indices

    return evaluate_count_accuracy


# Combine z_pres and z_where (as returned by the model and guide) into
# a single tensor, with size:
# [batch_size, num_steps, z_where_size + z_pres_size]
def latents_to_tensor(z):
    return jnp.stack(
        [
            jnp.concatenate((z_where, z_pres.reshape(-1, 1)), 1)
            for z_where, z_pres in zip(*z)
        ]
    ).transpose(1, 0, 2)

In [ ]:
##################
# Visualization  #
##################


def bounding_box(z_where, x_size):
    """This doesn't take into account interpolation, but it's close
    enough to be usable."""
    w = x_size / z_where.s
    h = x_size / z_where.s
    xtrans = -z_where.x / z_where.s * x_size / 2.0
    ytrans = -z_where.y / z_where.s * x_size / 2.0
    x = (x_size - w) / 2 + xtrans  # origin is top left
    y = (x_size - h) / 2 + ytrans
    return (x, y), w, h


z_obj = namedtuple("z", ["s", "x", "y", "pres"])


# Map a tensor of latents (as produced by latents_to_tensor) to a list
# of z_obj named tuples.
def tensor_to_objs(latents):
    return [[z_obj._make(step) for step in z] for z in latents]


def visualize_model(model, guide):
    def reconstruct_digits(key, data, params):
        decoder, rnn, encoder, predict, bl_rnn, bl_predict = params
        data_chmp = genjax.choice_map({"obs": data})
        key1, key2 = jax.random.split(key)
        tr = guide.simulate(
            key1, (data_chmp, rnn, encoder, predict, bl_rnn, bl_predict)
        )
        _, tr = model.importance(key2, tr, (decoder,))
        reconstruction = tr.get_retval()
        z_where = [tr[f"z_where_{i}"] for i in range(3)]
        z_pres = [tr[f"z_pres_{i}"] for i in range(3)]
        return reconstruction, (z_where, z_pres)

    batch_reconstruct_digits = jax.jit(
        jax.vmap(reconstruct_digits, in_axes=(0, 0, None))
    )

    def visualize(key, params, examples_to_viz):
        keys = jax.random.split(key, examples_to_viz.shape[0])
        recons, z = batch_reconstruct_digits(keys, examples_to_viz, params)
        z_wheres = tensor_to_objs(latents_to_tensor(z))
        draw_many(examples_to_viz.reshape(-1, 50, 50), z_wheres, title="Original")
        draw_many(recons, z_wheres, title="Reconstruction")

    return visualize


def colors(k):
    return ["r", "g", "b"][k % 3]


def draw_one(img, z):
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img, cmap="gray_r")
    for k, z in enumerate(z):
        if z.pres > 0:
            (x, y), w, h = bounding_box(z, img.shape[0])
            plt.gca().add_patch(
                Rectangle(
                    (x, y), w, h, linewidth=1, edgecolor=colors(k), facecolor="none"
                )
            )


def draw_many(imgs, zs, title):
    plt.figure(figsize=(8, 1.9))
    plt.title(title)
    plt.tick_params(left=False, labelleft=False, bottom=False, labelbottom=False)
    plt.box(False)
    for i, (img, z) in enumerate(zip(imgs, zs)):
        plt.subplot(1, len(imgs), i + 1)
        draw_one(img, z)
    plt.show()

In [ ]:
params = (decoder, rnn, encoder, predict)
evaluate_accuracy = count_accuracy(mnist, true_counts, guide, batch_size=1000)

visualize_examples = mnist[5:10]
visualize = visualize_model(model, guide)

In [ ]:
batch_size = 64
learning_rate = 1.0e-4
baseline_learning_rate = 1.0e-2
num_epochs = 200


def svi_update(model, guide, guide_optimizer, baseline_optimizer):
    def batch_updater(
        key, params, baseline_params, guide_opt_state, baseline_opt_state, data_batch
    ):

        # First, ELBO grads.
        def elbo_grads(key, params, baseline_params, data):
            (decoder, rnn, encoder, predict) = params
            (bl_rnn, bl_predict) = baseline_params
            data = genjax.choice_map({"obs": data})

            @genjax.gen
            def inlined_guide(
                data: genjax.ChoiceMap,
                rnn: eqx.nn.LSTMCell,
                encoder: Encoder,
                predict: Predict,
            ):
                # Inline the guide into a new generative function.
                return guide.inline(data, rnn, encoder, predict, bl_rnn, bl_predict)

            objective = grasp.elbo(model, inlined_guide, data)

            loss, (
                (decoder_grads,),
                (_, rnn_grads, encoder_grads, predict_grads),
            ) = objective.value_and_grad_estimate(
                key, ((decoder,), (data, rnn, encoder, predict))
            )
            return loss, (decoder_grads, rnn_grads, encoder_grads, predict_grads)

        key, sub_key = jax.random.split(key)
        sub_keys = jax.random.split(sub_key, len(data_batch))

        # First, ELBO grads.
        loss, guide_grads = jax.vmap(elbo_grads, in_axes=(0, None, None, 0))(
            sub_keys, params, baseline_params, data_batch
        )

        guide_grads = jtu.tree_map(
            lambda v: -1.0 * jnp.mean(v, axis=0),
            guide_grads,
        )
        guide_updates, opt_state = guide_optimizer.update(
            guide_grads, guide_opt_state, params
        )
        params = optax.apply_updates(params, guide_updates)

        # Now, baseline grads.
        def baseline_grads(key, params, baseline_params, data):
            (decoder, rnn, encoder, predict) = params
            (bl_rnn, bl_predict) = baseline_params
            data = genjax.choice_map({"obs": data})

            @adevjax.adev
            def baseline_loss(bl_rnn, bl_predict):
                key = adevjax.reap_key()
                tr = guide.simulate(
                    key, (data, rnn, encoder, predict, bl_rnn, bl_predict)
                )
                qw = tr.get_score()
                chm = data.safe_merge(tr.strip())
                key = adevjax.reap_key()
                (_, pw) = model.assess(key, chm, (decoder,))
                Z = pw - qw
                bvs = jnp.array(tr.get_retval())
                return jnp.sum((bvs - Z) ** 2)

            loss, baseline_params_grad = adevjax.E(
                baseline_loss
            ).value_and_grad_estimate(
                key,
                baseline_params,
            )
            return loss, baseline_params_grad

        # Now, baseline grads.
        sub_keys = jax.random.split(key, len(data_batch))
        _, baseline_params_grads = jax.vmap(baseline_grads, in_axes=(0, None, None, 0))(
            sub_keys, params, baseline_params, data_batch
        )

        baseline_params_grads = jtu.tree_map(
            lambda v: jnp.mean(v, axis=0),
            baseline_params_grads,
        )
        baseline_updates, baseline_opt_state = baseline_optimizer.update(
            baseline_params_grads, baseline_opt_state, baseline_params
        )
        baseline_params = optax.apply_updates(baseline_params, baseline_updates)

        # From ELBO loss.
        loss = jnp.mean(loss)

        return params, baseline_params, guide_opt_state, baseline_opt_state, loss

    return batch_updater


guide_adam = optax.adam(learning_rate)
baseline_adam = optax.adam(baseline_learning_rate)
svi_updater = svi_update(model, guide, guide_adam, baseline_adam)
train_init, train_fetch = data_loader(jnp.array(mnist), batch_size)
num_batch, train_idx = train_init()


@jax.jit
def epoch_train(
    guide_opt_state, baseline_opt_state, params, baseline_params, key, train_idx
):
    def body_fn(carry, xs):
        idx, params, baseline_params, guide_opt_state, baseline_opt_state, loss = carry
        updater_key = jax.random.fold_in(key, idx)
        batch = train_fetch(idx, train_idx)
        (
            params,
            baseline_params,
            guide_opt_state,
            baseline_opt_state,
            loss,
        ) = svi_updater(
            updater_key,
            params,
            baseline_params,
            guide_opt_state,
            baseline_opt_state,
            batch,
        )
        idx += 1
        return (
            idx,
            params,
            baseline_params,
            guide_opt_state,
            baseline_opt_state,
            loss,
        ), loss

    idx = 0
    (
        _,
        params,
        baseline_params,
        guide_opt_state,
        baseline_opt_state,
        _,
    ), losses = jax.lax.scan(
        body_fn,
        (idx, params, baseline_params, guide_opt_state, baseline_opt_state, 0.0),
        None,
        length=num_batch,
    )
    return params, baseline_params, guide_opt_state, baseline_opt_state, losses


# Train.
key = jax.random.PRNGKey(314159)
params = (decoder, rnn, encoder, predict)
baseline_params = (bl_rnn, bl_predict)
guide_opt_state = guide_adam.init(params)
baseline_opt_state = baseline_adam.init(baseline_params)
losses = []
accuracy = []

# Warm up.
_ = epoch_train(
    guide_opt_state, baseline_opt_state, params, baseline_params, key, train_idx
)

t0 = time.perf_counter()
acc_time = 0.0
for i in range(0, num_epochs):
    key, sub_key = jax.random.split(key)
    num_batch, train_idx = train_init()
    start = time.perf_counter() - t0
    params, baseline_params, guide_opt_state, baseline_opt_state, loss = epoch_train(
        guide_opt_state, baseline_opt_state, params, baseline_params, sub_key, train_idx
    )
    stop = time.perf_counter() - t0
    acc_time += stop - start
    print(f"Epoch={i}, total_epoch_step_time={acc_time:.2f}, loss={jnp.mean(loss):.2f}")
    if i % 20 == 0:
        acc, counts, error_ix = evaluate_accuracy(
            sub_key, (*params[1:], *baseline_params)
        )
        accuracy.append(acc)
        print("accuracy={}, counts={}".format(acc, counts))
    if i % 20 == 0:
        visualize(sub_key, (*params, *baseline_params), visualize_examples)

In [ ]:
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(np.linspace(0, num_epochs, len(losses)), losses)
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.subplot(1, 2, 2)
plt.plot(np.linspace(0, num_epochs, len(accuracy)), accuracy)
plt.xlabel("Epoch")
plt.ylabel("Accuracy");